In [1]:
library(ggplot2)
library(reshape2)
library(dplyr)
library(stringr)
theme_set(theme_bw())
options(repr.plot.width=7, repr.plot.height=4)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
identities = read.delim('identities.tsv', sep='\t')
identities$quality = as.logical(identities$quality)
identities$restrict = as.logical(identities$restrict)

# Introduction

Here, I measure base frequencies and tabulate identity elements by clade. Biologists in the past have named things as identity elements if 100% of tRNAs have this element. I'll relax those restrictions to 99% among the quality and species-restricted tRNAs.

Terminology:
- An **identity element** is a base or base class (G, purine, etc.) located at a specific position.
- **Consensus** identity elements for a clade means that all subclades contain this identity element.
- **Diverged** positions for a clade means that all subclades contain an IDE at that position, even if the IDE differs. Of course, the IDE itself could be "zoomed out", to a lower specificity, and then the position wouldn't be described as diverged anymore. For example, if archaea and bacteria have A41 but eukarya have G41, then position 41 is diverged if you consider A41 and G41 to be separate IDEs, but not diverged if you call it as R41.
- **Unique** identity elements for a subclade means that all sister subclades do not have an IDE at this position.
- We could potentially have a case where certain subclades have an IDE at a position while others do not. These will be known as **shared** identity elements among two or more unrelated subclades. 

# Consensus eukaryotic identity elements

I will start with the easiest case: IDEs that are almost perfectly conserved among all eukaryotes.